In [1]:
from setup_general import *

/home/till/projects/uni/data science/museum_item_classification/setup_general.py:93: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_intermediate_ready = pd.read_csv('./data/general/combined_intermediate_ready.csv', index_col='id', dtype={'type': str})


# setup data for classifier

In [4]:
data = train_est_prepared.copy()

data['type'] = data.type.apply(lambda x: 'not_photo' if x != 'photo' else 'photo')

features = data.drop('type', axis=1)
labels = data.type
# at least xgboost cannot deal with string labels
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels)
labels = label_encoder.transform(labels)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=0)

# single rf classifier

In [22]:
rfc = RandomForestClassifier(n_estimators=100, random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.98      1.00      0.99      2756\n           1       0.93      0.76      0.84       184\n\n    accuracy                           0.98      2940\n   macro avg       0.96      0.88      0.91      2940\nweighted avg       0.98      0.98      0.98      2940\n'

In [5]:
#svm
from sklearn.svm import SVC
svm = SVC(random_state=0)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.81      0.83      0.82      2111\n           1       0.55      0.52      0.53       829\n\n    accuracy                           0.74      2940\n   macro avg       0.68      0.67      0.68      2940\nweighted avg       0.74      0.74      0.74      2940\n'

# sweeping

In [ ]:
project = 'rf'

# Define sweep config
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'split': {'values': [2, 5, 10, 100]},
        'depth': {'values': [3, 6, 10, 50, 100, 1000]},
        'leaf': {'values': [2, 6, 10, 50, 100, 1000, 5000]},
        'estimators': {'values': [200, 500, 1000, 2000]},
        'features': {'values': [None, 'sqrt', 'log2']},

     }
}

# Initialize sweep by passing in config. (Optional) Provide a name of the project.
sweep_id = wandb.sweep(sweep=sweep_configuration, project=project)

def main():
    run = wandb.init(project=project)

    # note that we define values from `wandb.config` instead 
    # of defining hard values 
    split = wandb.config.split
    depth = wandb.config.depth
    leaf = wandb.config.leaf
    estimators = wandb.config.estimators
    feat = wandb.config.features
    

    # -------------------------- usual training code starts here  -------------------------------------
    
    rfc = RandomForestClassifier(n_estimators=estimators, max_depth=depth, min_samples_leaf=leaf, max_features=feat, min_samples_split=split, random_state=42)
    rfc.fit(X_train, y_train)

    y_pred = rfc.predict(X_test)
    val_acc = accuracy_score(y_test, y_pred)
    
    y_pred = rfc.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred)

    print(train_acc, val_acc)

    # -------------------------- ends here  -------------------------------------
    

    wandb.log({
      'train_acc': train_acc,
      'val_acc': val_acc,
    })

# Start sweep job.
wandb.agent(sweep_id, function=main)

# save model

In [5]:
pickle.dump(rfc, open('./models/rf/train_prep_full_best', 'wb'))
loaded_model = pickle.load(open('./models/rf/first_try', 'rb'))

# submission from model

In [ ]:
test_set = test_prep.drop('type', axis=1)
submission = pd.DataFrame({'id': test_set.index ,'type': rfc.predict(test_set)})
submission = submission.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())
submission.to_csv('submissions/some_rf_model.csv', index=False)